### 크롤링으로  업종별 코드 가져오기

In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import pandas as pd

import time
import urllib.request
from selenium.webdriver import Chrome
import json
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt

# selenium을 위한 chrome 브라우저 켜기
browser  = Chrome()
#browser  = IE()
browser.maximize_window()

annual_date = ['2015/12', '2016/12', '2017/12', '2018/12', '2019/12', '2020/12','2021/12', '2022/12']
# 각각 지표들 데이터프레임에 넣기 위한 초기 선언, (df_NI(Net Income) : 당기순이익을 뜻함)

url_up = 'https://finance.naver.com/sise/sise_group.nhn?type=upjong'

## 밑에는 업종별 코드를 반환하기 위한 코드 
res = requests.get(url_up)
html = res.text
soup = BeautifulSoup(html,'html.parser')

upjong = {}
rows = soup.select('td > a')

for r in range(len(rows)):
    upjong[soup.select('td > a')[r]['href'][43:]] = soup.select('td > a')[r].text
upjong_code = list(upjong.keys())[:-1]
# 기타 업종 제외 -> 재무제표 제공 X 
del upjong_code[45]
print(upjong_code)

['205', '144', '233', '33', '232', '229', '136', '138', '200', '235', '189', '217', '183', '197', '230', '222', '216', '220', '173', '185', '215', '204', '211', '212', '193', '171', '25', '219', '166', '35', '198', '42', '109', '176', '140', '164', '199', '210', '36', '194', '20', '53', '182', '228', '208', '192', '187', '62', '154', '168', '202', '162', '214', '134', '203', '184', '38', '191', '201', '12', '213', '146', '206', '227', '145', '44', '196', '174', '161', '167', '221', '226', '231', '234', '218', '190', '236', '177']


In [2]:
# 업종코드를 입력하면 --> 해당 기업 코드가 나오는 함수. 
def upjong_url(code):
    tickers = {}
    url_tmp = "https://finance.naver.com/sise/sise_group_detail.nhn?type=upjong&no=%s"
    url = url_tmp % (code)
    res = requests.get(url)
    html = res.text
    
    soup = BeautifulSoup(html,"html.parser")
    rows = soup.select('td > div')

    for r in rows[1:]:
        tickers[r.select('a')[0]['href'][20:]] = r.text
    a = [k for k in tickers.keys() if tickers[k][-2]=='우' or tickers[k][-3]=='우']
    # 우량주 종목 제외    
    for i in range(len(a)):
        del tickers[a[i]]

    code = list(tickers.keys())
    com = list(tickers.values())
    return code,com
#회사 코드를 사용해서 url완성하는 함수
def ttt(xx):
    name = xx
    base_url = 'https://finance.naver.com/item/coinfo.nhn?code='+ name + '&target=finsum_more'
    return base_url
# 종목코드를 통해 재무제표 크롤링
def stock_crawler(code):
    #code = 종목번호
    name = code
    base_url = 'https://finance.naver.com/item/coinfo.nhn?code='+ name + '&target=finsum_more'
    
    browser.get(base_url)
    #frmae구조 안에 필요한 데이터가 있기 때문에 해당 데이터를 수집하기 위해서는 frame구조에 들어가야한다.
    browser.switch_to.frame(browser.find_element_by_id('coinfo_cp'))
    
    #재무제표 "연간" 클릭하기
    browser.find_elements_by_xpath('//*[@class="schtab"][1]/tbody/tr/td[3]')[0].click()

    html0 = browser.page_source
    html1 = BeautifulSoup(html0,'html.parser')
    
    #기업명 뽑기
    title0 = html1.find('head').find('title').text
    #print(title0.split('-')[-1])
    
    html22 = html1.find('table',{'class':'gHead01 all-width','summary':'주요재무정보를 제공합니다.'})
    
    #date scrapy
    thead0 = html22.find('thead')
    tr0 = thead0.find_all('tr')[1]
    th0 = tr0.find_all('th')
    
    date = []
    for i in range(len(th0)):
        date.append(''.join(re.findall('[0-9/]',th0[i].text)))
    
    #columns scrapy
    tbody0 = html22.find('tbody')
    tr0 = tbody0.find_all('tr')
    
    col = []
    for i in range(len(tr0)):

        if '\xa0' in tr0[i].find('th').text:
            tx = re.sub('\xa0','',tr0[i].find('th').text)
        else:
            tx = tr0[i].find('th').text

        col.append(tx)
    
    #main text scrapy
    td = []
    for i in range(len(tr0)):
        td0 = tr0[i].find_all('td')
        td1 = []
        for j in range(len(td0)):
            if td0[j].text == '':
                td1.append('0')
            else:
                td1.append(td0[j].text)

        td.append(td1)
    
    td2 = list(map(list,zip(*td)))
    
    return pd.DataFrame(td2,columns = col,index = date).T
# numpy -> pandas 데이터프레임으로 변환하는 사용자 정의 함수
def trans_df(ar,com):
    #df = np.array([])
    metrics = ['PER(배)','PBR(배)','EPS(원)','ROE(%)','ROA(%)','당기순이익','매출액','영업이익','발행주식수(보통주)']
    ar = np.resize(ar,(int(len(com)),9))
    #ar.resize((int(len(com)),6))
    df = pd.DataFrame(data = ar,index=com,columns = metrics)
    return df

In [4]:
tickers = {}
for up in upjong_code:
    code, com = upjong_url(up)
    for i in range(len(code)):
        tickers[code[i]] = com[i]
        
#시가총액 구하기 단위 억원
from marcap import marcap_data
df = marcap_data('2017-12-28')
Mc = pd.DataFrame(df['Marcap'].values/10**8,index=df['Name'].values,columns=['시가총액'])        

In [12]:
import json

tickers = {}
for up in upjong_code:
    code, com = upjong_url(up)
    for i in range(len(code)):
        tickers[code[i]] = com[i]
# CONVERT dictionary to json using json.dump

json_val = json.dumps(tickers)

with open('./tickers.json','w') as f:
    json.dump(json_val,f)

with open('./tickers.json','r') as f:
    obj3 = json.load(f)
tickers = json.loads(obj3)

In [97]:
code

['029530', '018290']

In [218]:
# res가 업종별 5순위 종목을 누적해서 데이터프레임에 저장 업종코드 res라는 전혀 다른 의미.

res1,res2,res3 = pd.DataFrame([]),pd.DataFrame([]),pd.DataFrame([])
com,toplist = [],[]
# 지표 기준 년도 
#sc ='2019/12'
p = re.compile('2017.[0-9][0-9]')
# 지표 기준 년도 
#sc ='2019/12'
i = 1
for up in upjong_code[:10]:
    # 업종별로 회사 코드를 받음, 전자장비및기기업종 -> 78개 회사 리스트를 받는다.
    # 몇 번째 업종 불러오는지 표시하기 위한 것으로 주석처리해도 무방
    print('진행상황 : {}/{}'.format(i,len(upjong_code)))
    df_com = np.array([])
    code, _ = upjong_url(up)
    #print(code)
    for c in code:
        # 재무제표를 종목별로 불러와 지표 저장
        df = stock_crawler(c)
        #print(df)
        df = df.apply(pd.to_numeric,errors='coerce')
        df.fillna(value=0,inplace=True)
        for d in list(df.columns):
            if p.findall(d)!= []:
                sc = p.findall(d)[0]
            else:
                sc = df.columns[0]
        if df.loc[['PER(배)','PBR(배)','EPS(원)','ROE(%)','ROA(%)','당기순이익','매출액','영업이익','발행주식수(보통주)'],sc].isnull().sum().sum() == 0 and df[sc].sum().sum()>0: 
        
            df_com = np.hstack([df_com,df.loc[['PER(배)','PBR(배)','EPS(원)','ROE(%)','ROA(%)','당기순이익','매출액','영업이익','발행주식수(보통주)'],sc].values])
            com.append(tickers[c])
    #print(com)    
    # 데이터프레임으로 변환
    df_com = trans_df(df_com,com)
    print(df_com)
    # 지표별로 랭크 구하여 합산하기
    
    #df_com['upjong_rank'] = (df_com[['PER(배)','PBR(배)']].rank().sum(axis=1)+df_com[['ROE(%)','ROA(%)','EPS(원)','당기순이익']].rank(method='max').sum(axis=1)).rank()
    #df_com.sort_values(by='upjong_rank',inplace=True)
    #toplist.append(list(df_com[df_com['upjong_rank']<=5].index))
    #res = pd.concat([res,df_com[df_com['upjong_rank']<=5]])
    res1 = pd.concat([res1,df_com])
    i+=1

new_ind = []
for i in range(len(res1)):
    new_ind.append(res1.index[i][:-2])
res1.index = new_ind
res1.drop(set(res1.index)-set(Mc.index),inplace=True)
res1['시가총액'] = Mc.loc[set(res1.index) &  set(Mc.index)]
res1['PSR'] = res1['시가총액'] / res1['매출액']
res1['POR'] = res1['시가총액'] / res1['영업이익']
res1 = res1[res1>=0]
res1.dropna(inplace=True)
res2,res3 = res1.copy(),res1.copy()
res1['total_rank'] = res1[['PBR(배)','PSR']].rank().sum(axis=1).rank()
res2['total_rank'] = res2[['PBR(배)','PSR','POR']].rank().sum(axis=1).rank()
res3['total_rank'] = (res3[['PER(배)','PBR(배)','PSR','POR']].rank().sum(axis=1)+res3[['EPS(원)', 'ROE(%)', 'ROA(%)', '당기순이익']].rank(ascending=False).sum(axis=1)).rank()  
#res['total_rank'] = (res[['PER(배)','PBR(배)']].rank().sum(axis=1)+res[['ROE(%)','ROA(%)','EPS(원)','당기순이익']].rank(ascending=False).sum(axis=1)).rank()
#res.sort_values(by='total_rank',inplace=True)
res1.sort_values(by='total_rank',inplace=True)
res2.sort_values(by='total_rank',inplace=True)
res3.sort_values(by='total_rank',inplace=True)

진행상황 : 1/78
          PER(배)  PBR(배)  EPS(원)  ROE(%)  ROA(%)  당기순이익    매출액  영업이익  \
윌링스 *       0.00    0.00   575.0   18.91    7.55   21.0  420.0  25.0   
에스에너지 *    28.29    0.94   281.0    3.63    1.26   32.0    0.0  56.0   
SDN *      67.26    1.24    27.0    1.71    0.81    8.0  482.0  55.0   
서남 *        0.00    0.00     0.0  -47.75  -25.84  -55.0   39.0  -6.0   
젬백스지오 *     0.00    3.92  -335.0    0.00    0.00  -84.0  448.0 -52.0   
파루 *        9.41    2.04   468.0   25.31   11.40   97.0    0.0  98.0   
코드네이처 *     0.00    3.21   -37.0   -2.90   -0.97   -3.0  109.0  -1.0   
제이씨케미칼 *   15.31    1.04   202.0    6.28    2.25   27.0    0.0  40.0   

          발행주식수(보통주)  
윌링스 *            0.0  
에스에너지 *          0.0  
SDN *            0.0  
서남 *             0.0  
젬백스지오 *          0.0  
파루 *             0.0  
코드네이처 *          0.0  
제이씨케미칼 *         0.0  
진행상황 : 2/78
              PER(배)  PBR(배)  EPS(원)  ROE(%)  ROA(%)  당기순이익    매출액   영업이익  \
윌링스 *          19.63    1.33     0.0    7.0

In [224]:
res1.to_excel('PBR+PSR.xlsx')
res2.to_excel('PBR+PSR+POR.xlsx')
res3.to_excel('all case.xlsx')
#df_com['PBR(배)'].drop_duplicates()

In [105]:
# ind_p = re.compile('')
#         for d in list(df.columns):
#             if p.findall(d)!= []:
#                 sc = p.findall(d)[0]
#             else:
#                 sc = df.columns[0]
#res.index[5][:-2]
#res.index[4]
#Mc.loc[list(res.index)[3].split(' ')]

#len(np.unique(np.array(res.index)))

In [126]:
Mc.loc[set(Mc.index)  set(res.index)]
#Mc.loc[(res.index),'시가총액']

SyntaxError: invalid syntax (<ipython-input-126-013eb6d4e88c>, line 1)

In [199]:
res1['total_rank'] = res1[['PBR(배)','PSR']].rank().sum(axis=1).rank()
res2['total_rank'] = res2[['PBR(배)','PSR','POR']].rank().sum(axis=1).rank()
res3['total_rank'] = (res3[['PER(배)','PBR(배)','PSR','POR']].rank().sum(axis=1)+res3[['EPS(원)', 'ROE(%)', 'ROA(%)', '당기순이익']].rank(ascending=False).sum(axis=1)).rank()
res1.sort_values(by='total_rank',inplace=True)
res2.sort_values(by='total_rank',inplace=True)
res3.sort_values(by='total_rank',inplace=True)

In [200]:
res3
#res1[['EPS(원)', 'ROE(%)', 'ROA(%)', '당기순이익']]
#.rank(ascending=False).sum(axis=1)

,PER(배),PBR(배),EPS(원),ROE(%),ROA(%),당기순이익,매출액,영업이익,발행주식수(보통주),upjong_rank,시가총액,PSR,POR,total_rank
서전기전,0.0,0.0,274.0,29.49,14.56,23.0,345.0,32.0,0.0,14.5,3.297585e+10,9.558218e+07,1.030495e+09,1.0
피앤씨테크,0.0,0.0,274.0,29.49,14.56,23.0,345.0,32.0,0.0,14.5,5.405171e+10,1.566716e+08,1.689116e+09,2.0
SDN,0.0,0.0,274.0,29.49,14.56,23.0,345.0,32.0,0.0,14.5,5.908442e+10,1.712592e+08,1.846388e+09,3.0
지엔씨에너지,0.0,0.0,274.0,29.49,14.56,23.0,345.0,32.0,0.0,14.5,6.681289e+10,1.936606e+08,2.087903e+09,4.0
제이씨케미칼,0.0,0.0,274.0,29.49,14.56,23.0,345.0,32.0,0.0,14.5,7.882272e+10,2.284717e+08,2.463210e+09,5.0
제룡산업,0.0,0.0,274.0,29.49,14.56,23.0,345.0,32.0,0.0,14.5,8.100000e+10,2.347826e+08,2.531250e+09,6.0
세명전기,0.0,0.0,274.0,29.49,14.56,23.0,345.0,32.0,0.0,14.5,8.690220e+10,2.518904e+08,2.715694e+09,7.0
제룡전기,0.0,0.0,274.0,29.49,14.56,23.0,345.0,32.0,0.0,14.5,9.508946e+10,2.756216e+08,2.971546e+09,8.0
에스에너지,0.0,0.0,274.0,29.49,14.56,23.0,345.0,32.0,0.0,14.5,9.787513e+10,2.836960e+08,3.058598e+09,9.0
비츠로테크,0.0,0.0,274.0,29.49,14.56,23.0,345.0,32.0,0.0,14.5,1.089751e+11,3.158699e+08,3.405473e+09,10.0


In [133]:
res1.drop(set(res1.index)-set(Mc.index),inplace=True)
res1['시가총액'] = Mc.loc[set(res1.index) &  set(Mc.index)]
res1['PSR'] = res1['시가총액'] / res1['매출액']
res1['POR'] = res1['시가총액'] / res1['영업이익']

,PER(배),PBR(배),EPS(원),ROE(%),ROA(%),당기순이익,매출액,영업이익,발행주식수(보통주),upjong_rank
엔에스엔,0.00,2.12,-662.0,-34.05,-22.88,-88.0,186.0,-27.0,0.0,1.0
알톤스포츠,0.00,1.33,-304.0,-6.59,-4.92,-36.0,623.0,-24.0,0.0,2.0
라이브플렉스,89.07,1.38,9.0,1.56,-4.11,-23.0,422.0,-41.0,0.0,3.0
이녹스,18.24,1.24,824.0,7.44,2.88,68.0,0.0,106.0,0.0,6.0
헝셩그룹,0.00,0.00,469.0,31.44,18.32,287.0,0.0,395.0,0.0,7.0
오로라,24.35,1.72,577.0,7.49,3.16,62.0,0.0,135.0,0.0,8.0
손오공,14.04,4.39,389.0,37.26,15.00,93.0,0.0,104.0,0.0,9.0
삼천리자전거,18.29,2.14,943.0,13.20,10.09,125.0,0.0,150.0,0.0,10.0
헝셩그룹,0.00,0.21,0.0,-64.00,-24.29,-46.0,155.0,-48.0,0.0,2.0
엔에스엔,0.00,4.52,-267.0,-40.64,-30.14,-61.0,171.0,37.0,0.0,5.0


In [23]:
import FinanceDataReader as fdr
#stock_list = tickers
df_list = [fdr.DataReader(code, '2017-12-28','2017-12-29')['Close'] for code in list(tickers.keys())]
#price = pd.concat(df_list, axis=1)
#price.columns = [name for name, code in zip(list(stock_list.values()),list(tickers.keys()))]
#price.info()

In [24]:
price = pd.concat(df_list, axis=1)
price.columns = [name for name, code in zip(list(stock_list.values()),list(tickers.keys()))]
price.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1 entries, 2017-12-28 to 2017-12-28
Columns: 2940 entries, 신테카바이오 * to 일진디스플 
dtypes: float64(512), int64(2428)
memory usage: 23.0 KB


PSR = 시가총액 / 매출액  
POR = 시가총액 / 영업이익


In [22]:
df

,,,,2018/12,2019/12,2020/12,2021/12,2022/12
매출액,0,0,0,0,700.00,0.00,0.00,0.00
영업이익,0,0,0,0,102.00,444.00,750.00,0.00
영업이익(발표기준),0,0,0,0,102.00,0.00,0.00,0.00
세전계속사업이익,0,0,0,0,68.00,412.00,705.00,977.00
당기순이익,0,0,0,0,45.00,298.00,512.00,711.00
당기순이익(지배),0,0,0,0,45.00,298.00,512.00,711.00
당기순이익(비지배),0,0,0,0,0.00,0.00,0.00,0.00
자산총계,0,0,0,0,0.00,0.00,0.00,0.00
부채총계,0,0,0,0,0.00,0.00,0.00,0.00
자본총계,0,0,0,0,0.00,0.00,0.00,0.00


In [164]:
df_com['발행주식수(보통주)'].values * price[list(df_com.index)].values

array([], shape=(1, 0), dtype=float64)

In [172]:
ex_code = []
for c in tickers.keys():
    df = stock_crawler(c)
    if p.findall(d)!= []:
        sc = p.findall(d)[0]
    else:
        sc = df.columns[0]
    if df.loc['발행주식수(보통주)',sc] != 0:
        ex_code.append(c)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().